# SQL. Зачем нужен этот интенсив?

Для правильной работы сайта или приложения нужны не только файлы с кодом страниц, но и базы данных. 

Для взаимодействия с базами данных используются системы управления базами данных.  В качестве языка запросов к базе данных используется SQL.

В этом модуле вы узнаете о базах данных и СУБД, их разновидностях и основных отличиях, а так же попробуете поработать с ними.

## Инициализация базы

Прежде, чем начать работать с базой данных, ее нужно создать и наполнить данными. Сейчас на ваших компьютерах уже развернут сервер баз данных, и на нём подготовлено пространство для таблиц (т.н. схема базы данных). Заполнение ее - достаточно долгий процесс, поэтому займемся этим сразу же, чтобы можно было параллельно дальше изучать теорию.

## Задание 1

Исполните следующий скрипт, который заполняет БД. Некоторые библиотеки придется доставить самостоятельно.

In [15]:
!python3.10 -m pip install sqlalchemy
!python3.10 -m pip install sqlalchemy.orm

ERROR: Could not find a version that satisfies the requirement sqlalchemy.ext.automap (from versions: none)
ERROR: No matching distribution found for sqlalchemy.ext.automap


In [17]:
!python3.10 -m pip install psycopg2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.3/384.3 kB 2.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for psycopg2: filename=psycopg2-2.9.5-cp310-cp310-macosx_11_0_x86_64.whl size=143731 sha256=e4cf063017d5afc02f786c5282201943e0e7889e6c7b5e9fbab1f6aeefe104f6
  Stored in directory: /Users/klotzgal/Library/Caches/pip/wheels/b4/83/91/37687788ee41584610b6db5b85f98ed987fffb2ca193558264
Successfully built psycopg2


In [12]:
!python3.10 -m pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 4.4 MB/s eta 0:00:0000:0100:010m
  Attempting uninstall: pip
    Found existing installation: pip 23.0
    Uninstalling pip-23.0:
      Successfully uninstalled pip-23.0


In [20]:
!source Python_day_1/bin/activate

In [21]:
import datetime
import random
import itertools


from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, MetaData, Table, select, func, text
from sqlalchemy.ext.automap import automap_base

USERNAME = "klotzgal"  # замените на свой логин

connection_string = f"postgresql+psycopg2://{USERNAME}:@localhost:5432/{USERNAME}"
engine = create_engine(connection_string)
Session = sessionmaker(engine)

creation_query = """CREATE TABLE IF NOT EXISTS products (
    product_id SERIAL PRIMARY KEY,
    name varchar(255) NOT NULL,
    category varchar(255) NOT NULL,
    brand varchar(255) NOT NULL
);
CREATE TABLE IF NOT EXISTS stores (
    store_id SERIAL PRIMARY KEY,
    address VARCHAR(255) NOT NULL,
    region INTEGER NOT NULL
);
CREATE TABLE IF NOT EXISTS customers (
    customer_id SERIAL PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    surname VARCHAR(255),
    birth_date DATE NOT NULL
);
CREATE TABLE IF NOT EXISTS prices (
    price_id SERIAL UNIQUE,
    product_id INTEGER NOT NULL,
    price INTEGER NOT NULL,
    start_date DATE NOT NULL,
    end_date DATE,
    FOREIGN KEY (product_id) REFERENCES products(product_id),
    CONSTRAINT product_start_end PRIMARY KEY (product_id, start_date, end_date)
);
CREATE TABLE IF NOT EXISTS sales (
    sale_id SERIAL PRIMARY KEY,
    product_id INTEGER NOT NULL,
    store_id INTEGER NOT NULL,
    customer_id INTEGER NULL,
    sale_date DATE NOT NULL,
    FOREIGN KEY (product_id) REFERENCES products(product_id),
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
    FOREIGN KEY (store_id) REFERENCES stores(store_id)
);
"""

with Session() as session:
    session.execute(text(creation_query))
    session.commit()
    
meta = MetaData()
meta.reflect(engine)

customers = Table("customers", meta, autoload_with=engine)
products = Table("products", meta, autoload_with=engine)
stores = Table("stores", meta, autoload_with=engine)
prices = Table("prices", meta, autoload_with=engine)
sales = Table("sales", meta, autoload_with=engine)

Base = automap_base()
Base.prepare(autoload_with=engine)

Customer = Base.classes.customers
Product = Base.classes.products
Store = Base.classes.stores
Price = Base.classes.prices
Sale = Base.classes.sales

PERIOD_START = datetime.date(2019, 12, 1)
PERIOD_END = datetime.date(2021, 1, 31)


def create_customers():
    names = "Григорий Макс Клава Никита Бугага Бугага Евпатий Большой Анатолий Светлана Валерий Елизавета".split(" ")
    surnames = "Лепс Корж Кока Джигурда Бугага Коловрат Вассерман Лобода Меладзе Монеточка".split() + [None]
    birth_dates = [
        datetime.date(1990, 1, 1),
        datetime.date(1980, 2, 2),
        datetime.date(1970, 3, 3),
        datetime.date(1960, 4, 4),
        datetime.date(2000, 5, 5)
    ]
    
    with Session() as session:
        session.execute(text("TRUNCATE TABLE customers CASCADE"))
        for name, surname in itertools.product(names, surnames):
            customer = Customer(name=name, surname=surname, birth_date=random.choice(birth_dates))
            session.add(customer)
        session.commit()
        

def create_stores():
    streets = "Ленина Маркса Энгельса Андропова Пушкина Лермонтова Паустовского Чайковского".split()
    cities = "Заозерск Приозерск Подозерск Междуреченск Залесск Загорск Подгорск Пригорск Медногорск".split()
    houses = "1 2 3 4 5 6 7 8 Колотушкина".split()
    
    with Session() as session:
        session.execute(text("TRUNCATE TABLE stores CASCADE"))
        for city, street, house in itertools.product(cities, streets, houses):
            store = Store(region=cities.index(city), address=f"г. {city}, ул. {street}, д. {house}")
            session.add(store)
        session.commit()
        
        
def create_products():
    categories = "сосиски колбаса ветчина сыр картоха молоко пиво соль сахар нулевочка подгузник макароны спагетти салат огурцы томаты яблоки торт вафли конфеты рис".split()
    brands = "Вкусняха Объедайка БольшойЖивот Обжорка Кулинар Жора КраснаяЦена ДешевоИТочка Вкус Вкусовщина".split()
    types = "вес. уп.".split()
    
    with Session() as session:
        session.execute(text("TRUNCATE TABLE products CASCADE"))
        for category, brand, typ in itertools.product(categories, brands, types):
            product = Product(name=f"{category} {brand} {typ}", category=category, brand=brand)
            session.add(product)
        session.commit()
        

def _create_periods():
    start_date = PERIOD_START
    period = 6 if random.random() > 0.9 else 13
    end_date = start_date + datetime.timedelta(days=period)
    yield start_date, end_date
    while end_date <= PERIOD_END:
        start_date = end_date + datetime.timedelta(days=1)
        period = 6 if random.random() > 0.9 else 13
        end_date = start_date + datetime.timedelta(days=period)
        yield start_date, end_date

        
def create_prices():
    with Session() as session:
        session.execute(text("TRUNCATE TABLE prices CASCADE"))
        for product_id in map(lambda row: row[0], session.execute(select(Product.product_id).distinct())):
            random.seed(product_id)
            for start_date, end_date in _create_periods():
                price_value = random.randint(2500, 30000)
                price = Price(product_id=product_id, price=price_value, start_date=start_date, end_date=end_date)
                session.add(price)
            session.commit()
            
            
def _iter_dates():
    dt = PERIOD_START
    while dt <= PERIOD_END:
        yield dt
        dt += datetime.timedelta(days=1)
        
        
def _get_price(product_id, date, session=None):
    with session or Session() as session:
        query = (
            select(Price.price)
            .filter(Price.product_id == product_id)
            .filter(Price.start_date <= date)
            .filter(Price.end_date >= date)
        )
        return session.execute(query).first()[0]


def create_sales():
    random.seed(42)
    with Session() as session:
        session.execute(text("TRUNCATE TABLE sales CASCADE"))
        product_ids = [pid[0] for pid in session.execute(select(Product.product_id)).all()]
        store_ids = [sid[0] for sid in session.execute(select(Store.store_id)).all()]
        customer_ids = [cid[0] for cid in session.execute(select(Customer.customer_id)).all()]
        customer_ids += [None] * 2 * len(customer_ids)
        for sale_date in _iter_dates():
            current_products = random.sample(product_ids, random.randint(40, 70))
            current_stores = random.sample(store_ids, random.randint(40, 50))
            current_customers = random.sample(customer_ids, random.randint(50, 60))
            print(sale_date)
            for pid in current_products:
                for sid in current_stores:
                    for cid in current_customers:
                        sale = Sale(product_id=pid, store_id=sid, customer_id=cid, sale_date=sale_date)
                        session.add(sale)
            session.commit()
            

create_customers()
create_stores()
create_products()
create_prices()
create_sales()

2019-12-01
2019-12-02
2019-12-03
2019-12-04
2019-12-05
2019-12-06
2019-12-07
2019-12-08
2019-12-09
2019-12-10
2019-12-11
2019-12-12
2019-12-13
2019-12-14
2019-12-15
2019-12-16
2019-12-17
2019-12-18
2019-12-19
2019-12-20
2019-12-21
2019-12-22
2019-12-23
2019-12-24
2019-12-25
2019-12-26
2019-12-27
2019-12-28
2019-12-29
2019-12-30
2019-12-31
2020-01-01
2020-01-02
2020-01-03
2020-01-04
2020-01-05
2020-01-06
2020-01-07
2020-01-08
2020-01-09
2020-01-10
2020-01-11
2020-01-12
2020-01-13
2020-01-14
2020-01-15
2020-01-16
2020-01-17
2020-01-18
2020-01-19
2020-01-20
2020-01-21
2020-01-22
2020-01-23
2020-01-24
2020-01-25
2020-01-26
2020-01-27
2020-01-28
2020-01-29
2020-01-30
2020-01-31
2020-02-01
2020-02-02
2020-02-03
2020-02-04
2020-02-05
2020-02-06
2020-02-07
2020-02-08
2020-02-09
2020-02-10
2020-02-11
2020-02-12
2020-02-13
2020-02-14
2020-02-15
2020-02-16
2020-02-17
2020-02-18
2020-02-19
2020-02-20
2020-02-21
2020-02-22
2020-02-23
2020-02-24
2020-02-25
2020-02-26
2020-02-27
2020-02-28
2020-02-29

# Определения

**База данных (БД)** - это взаимосвязанная информация (данные) об объектах, которая организована специальным образом и хранится на каком-либо носителе.

**Система управления базами данных (СУБД)** – это комплекс программно-языковых средств, позволяющих создать базы данных и управлять данными.

**SQL (Structured Query Language** – «Структурированный язык запросов») – это структурированный язык запросов, применяемый для создания, модификации и управления данными в базах данных.

![1.png](attachment:1.png)

Установка postgres на Линукс: https://www.digitalocean.com/community/tutorials/how-to-install-postgresql-on-ubuntu-20-04-quickstart

# Реляционные

![2.png](attachment:2.png)

# Сетевые

![3.png](attachment:3.png)

# Иерархические

![4.png](attachment:4.png)

# NoSQL

**NoSQL** - это подход к реализации масштабируемого хранилища (базы) информации с гибкой моделью данных, отличающийся от классических реляционных СУБД. 

![5.png](attachment:5.png)

# Классификация БД по среде хранения
## Вторичная память

Хранение данных во вторичной памяти, или традиционная средой постоянного хранения является периферийная энергонезависимая память (вторичная память) — как правило жёсткий диск. В оперативную память СУБД помещает лишь данные для текущей обработки

![6.png](attachment:6.png)

## Оперативная память

Хранение данных в оперативной памяти все данные на стадии исполнения находятся в оперативной памяти.

![7.png](attachment:7.png)

# ПО

https://www.pgadmin.org/download/

В утилите pgAdmin нам нужно будет подключиться к новому источнику данных (новое соединение с базой данных). Параметры этого источника указаны в переменной `connection_string` в коде выше. Это строка, описывающая всё, что связано с подключением к источнику данных. Ее формат: 

    postgresql://[user[:password]@][netloc][:port][/dbname]

Чтобы подключиться к базе через pgAdmin, подставьте соответствующие значения из connection_string в параметры подключения.
    

# Типы данных в SQL

- Числовые типы данных (целые и дробные числа):
    INT, NUMBER, FLOAT, DECIMAL
- Строковые типы данных:
    STRING, TEXT, VARCHAR
- Календарные типы данных (дата и время):
    DATE, TIME, DATETIME, TIMESTAMP

# Запросы 

## Операторы SELECT и FROM

Оператор **SELECT** осуществляет выборку из (**FROM**) базы данных. 
Вводить данные в запрос можно как строчными, так и прописными буквами. Пробелы и табуляция не повлиляют на запрос.
При этом столбцы и строки результирующего набора не упорядочены.

    SELECT *
    FROM <TABLE_NAME> ;

Чтобы упорядочить поля результирующего набора, их следует перечислить через запятую в нужном порядке после слова **SELECT** (вертикальная выборка):
    
    select <column_name1>, <column_name2>, …
    from <table_name> ;
    
 ### Примеры  
    
    SELECT
        NAME, 
        CATEGORY
    FROM 
        PRODUCTS ;

    SELECT store_id, address
    FROM stores;

    SELECT *
    FROM customer ;
    
Выборка может содержать дубликаты строк в том случае, если она не содержит потенциального ключа, однозначно определяющего запись. Если требуется получить только уникальные строки (скажем, нас интересуют только уникальные комбинации названий и категорий продуктов), то можно использовать ключевое слово **DISTINCT**:
    
    SELECT DISTINCT name, category
    FROM products ;
    
Помимо **DISTINCT** может применяться также ключевое слово **ALL** (все строки), которое принимается по умолчанию.

## Оператор ORDER BY

Чтобы упорядочить строки результирующего набора, можно выполнить сортировку по любому количеству полей, указанных в предложении **SELECT**. Для этого используется предложение **ORDER BY** список полей. Поля в списке должды бать указаны в операторе **SELECT**. При этом в списке полей могут указываться как имена полей, так и их порядковые позиции в списке предложения **SELECT**. 
Сортировку можно проводить по возрастанию (параметр **ASC** принимается по умолчанию) или по убыванию (параметр **DESC**).
    
 ### Примеры
    
    SELECT DISTINCT name, brand 
    FROM products 
    ORDER BY brand DESC ;


Допустим, нам интересно сравнить, как изменялись цены на товар по месяцам, вне зависимости от года. Тогда нам нужно отсортировать цены по месяцу+дате появления цены.

    SELECT DISTINCT
        product_id,
        price,
	    EXTRACT(MONTH FROM start_date) price_month,
        EXTRACT(DAY FROM start_date) price_day
    FROM prices 
    ORDER BY product_id, price_month, price_day

## Оператор LIMIT и OFFSET

Оператор **LIMIT** отбирает указанную часть из запроса. 

Оператор **OFFSET** используется с оператором **LIMIT**, и исключает указанную часть, сдвигая нашу выборку ниже по таблице.

Например, выведем 5 самых дорогих товаров:

    SELECT product_id, price
    FROM prices
    WHERE start_date >= '2022-08-30'
    ORDER BY price DESC
    LIMIT 5 ;
    
С помощью **LIMIT** можно делать смещение (**OFFSET**).

Например, выведем не первые 5 самых дорогих товаров, а следующую пятерку.

    SELECT product_id, price 
    FROM prices 
    WHERE start_date >= '2022-08-30'
    ORDER BY price DESC 
    LIMIT 5, 5 ;
    
    SELECT product_id, price 
    FROM prices 
    WHERE start_date >= '2022-08-30'
    ORDER BY price DESC 
    LIMIT 5 
    OFFSET 5 ;
    
## Оператор WHERE

Горизонтальную выборку реализует предложение **WHERE <предикат>**, которое записывается после предложения **FROM**. При этом в результирующий набор попадут только те строки из источника записей, для каждой из которых значение предиката равно **TRUE**. То есть предикат проверяется для каждой записи. 
Например, запрос «получить информацию какие продукты и в каких категориях выпускаются под маркой ***МАРКА***» можно сформулировать следующим образом:

    SELECT name, category
    FROM products
    WHERE brand = 'Вкус' ;
    
Символьные строки и константы типа дата/время записываются в апострофах.

В предложении **WHERE** можно использовать:

1. *Стандартные операции сравнения*

- «<» (меньше чем)

- «=» (равно)

- «>» (больше)

- «>=» (больше или равно)

- «<=» (меньше или равно)

- «<>» (не равно)

Выражения в предикатах сравнения могут содержать константы и любые поля из таблиц, указанных в предложении **FROM**. 
    
2. *Числовой диапазон*

- **BETWEEN** … **AND** …    Число входит в числовой диапазон (включая диапазон, указанный в операторе)

- **NOT BETWEEN** … **AND** …    Число не входит в числовой диапазон (включая диапазон, указанный в операторе)

3. *Числовой список*
   
- **IN** (…)    Включение числа в числовой список

- **NOT IN** (…)   Исключение числа из числового списка

4. *Операторы* ***AND*** *и* ***OR***

- Оператор **AND** показывает результат при выполнении одновременно нескольких условий.

- Оператор **OR** показывает результат при выполнении хотя бы одного из нескольких условий.
 
5. *Предикат* ***LIKE*** *и* ***NOT LIKE***

- **LIKE** 'abc%'    Любые строки, которые начинаются с букв «abc»

- **LIKE** 'abc_'    Строки длиной строго 4 символа, причем первыми символами строки должны быть «abc»

- **LIKE** '%z'    Любая последовательность символов, которая обязательно заканчивается символом «z»

- **LIKE** '%Rostov%'    Любая последовательность символов, содержащая слово «Rostov» в любой позиции строки

- **LIKE** '% % %'    Текст, содержащий не менее 2-х пробелов, например, "World Wide Web"

Если искомая строка содержит трафаретный символ, то следует задать управляющий символ в предложении **ESCAPE**. Этот управляющий символ должен использоваться в образце перед трафаретным символом, сообщая о том, что последний следует трактовать как обычный символ. Например, если в некотором поле следует отыскать все значения, содержащие символ «_», то шаблон ‘%_%’ приведет к тому, что будут возвращены все записи из таблицы. В данном случае шаблон следует записать следующим образом:

**LIKE** '%#_%' ESCAPE '#'

Истинностное значение предиката LIKE присваивается в соответствии со следующими правилами:
 
- если либо проверяемое значение, либо образец, либо управляющий символ есть **NULL**, истинностное значение нельзя установить;

- в противном случае, если проверяемое значение и образец имеют нулевую длину, истинностное значение равно **TRUE**;

- в противном случае, если проверяемое значение соответствует шаблону, то предикат **LIKE** равен **TRUE**;

- если не соблюдается ни одно из перечисленных выше условий, предикат **LIKE** равен **FALSE**.

6. *Отбор и исключение* ***NULL***

- Значение NULL – пустая ячейка.

- Отбор и исключение пустых строк производится следующим образом: **IS/IS NOT NULL**. Важно помнить, что всегда выполняется **NULL != NULL**!!!

### Примеры

    SELECT *
    FROM sales
    WHERE store_id IN ('592', '234')
    AND customer_id is null ;

    SELECT *
    FROM prices
    WHERE start_date BETWEEN '2022-01-04' AND '2022-12-27'
    OR price >= 10000 ;

    SELECT name, surname
    FROM customers
    WHERE customer_id = 28 ;

    
    SELECT *
    FROM stores
    WHERE store_id IN (1,2,3,4,5,6,7,8,9,10)
    OR region IN (0,2) ;

    SELECT *
    FROM products
    WHERE brand LIKE 'К%'
    AND name NOT LIKE '%вес%';
    
## Агрегатные функции

- **COUNT(*)**	  Возвращает количество строк источника записей
- **COUNT()**       Возвращает количество полученных значений
- **AVG()**         Возвращает среднее арифметическое значение
- **SUM()**         Возвращает сумму значений
- **MIN()**         Возвращает минимальное значение
- **MAX()**         Возвращает максимальное значение
    
При этом функции **COUNT**, **MIN** и **MAX** применимы к данным любого типа, в то время как **SUM** и **AVG** используются только для данных числового типа. Разница между функцией **COUNT(*)** и **COUNT**(имя столбца | выражение) состоит в том, что вторая (как и остальные агрегатные функции) при подсчете не учитывает **NULL** -значения.

Агрегатные функции могут использоваться отдельно от оператора **GROUP BY**, но только в том случае, если применяются к единственному атрибуту в таблице или сразу ко всей таблице. Во всех остальных случаях использование **GROUP BY** необходимо.

Агрегатные функции НИКОГДА не применяются в условиях **WHERE**. Для их применения в условии используется ИЛИ вложенный запрос, ИЛИ оператор вспомогательного условия **HAVING**.

### Примеры
    
    SELECT count(*)
    FROM products ;
    
    SELECT count(distinct region)
    FROM stores ;
    
    SELECT AVG(price)
    FROM prices
    where product_id = 184 ;
    
    SELECT min(price), max(price), avg(price)
    FROM prices ;
    
## Оператор GROUP BY

Предложение **GROUP BY** используется для определения групп выходных строк, к которым могут применяться агрегатные функции. Если это предложение отсутствует, и используются агрегатные функции, то все столбцы с именами, упомянутыми в **SELECT**, должны быть включены в агрегатные функции, и эти функции будут применяться ко всему набору строк, которые удовлетворяют предикату запроса. В противном случае все столбцы списка **SELECT**, не вошедшие в агрегатные функции, должны быть указаны в предложении **GROUP BY**. 

В результате чего все выходные строки запроса разбиваются на группы, характеризуемые одинаковыми комбинациями значений в этих столбцах.После чего к каждой группе будут применены агрегатные функции. 
Следует иметь в виду, что для **GROUP BY** все значения **NULL** трактуются как равные, то есть при группировке по полю, содержащему **NULL**-значения, все такие строки попадут в одну группу.

Если при наличии предложения **GROUP BY**, в предложении **SELECT** отсутствуют агрегатные функции, то запрос просто вернет по одной строке из каждой группы. Эту возможность, наряду с ключевым словом **DISTINCT**, можно использовать для исключения дубликатов строк в результирующем наборе.

### Примеры
    
    SELECT COUNT(product_id), category
    FROM products
    GROUP BY category ;
    
    SELECT
        sale_date,
        COUNT(distinct customer_id) as n_customers,
	    COUNT(product_id) as n_products
    FROM sales
    WHERE sale_date < '2019-02-01'
    GROUP BY sale_date;
    
## Оператор HAVING

Если предложение **WHERE** определяет предикат для фильтрации строк, то предложение **HAVING** применяется после группировки для определения аналогичного предиката, фильтрующего группы по значениям агрегатных функций. 
Это предложение необходимо для проверки значений, которые получены с помощью агрегатной функции не из отдельных строк источника записей, определенного в предложении **FROM**, а из групп таких строк. Поэтому такая проверка не может содержаться в предложении **WHERE**.

Например, вывести список из id продуктов, у которых средняя цена за 2022 год около 17000:

    SELECT avg(price), product_id
    FROM prices
    WHERE start_date >= '2022-01-01' AND start_date <= '2022-12-31'
    GROUP BY product_id
    HAVING round(avg(price) , -3) = 17000 
    
Найти максимальную, минимальную и среднюю цену на товары при условии, что минимальная цена превышает 1000:

    SELECT min(price), max(price), avg(price)
    FROM prices
    HAVING min(price) > 1000 ;
    
## Аналитические функции

Аналитическая или оконная функция - функция вычисляется над некоторым окном строк. 

- **OVER** – перед вами аналитическая функция.
- **PARTITION BY** – разбивает строки на независимые группы.
- **ORDER BY** – задает порядок строк для вычисления функции.

Оконные функции делятся на:

1. *Агрегирующие*

- **COUNT(*)**	  Возвращает количество строк источника записей
- **COUNT()**       Возвращает сумму полученных значений
- **AVG()**         Возвращает среднее арифметическое значение
- **SUM()**         Возвращает сумму значений
- **MIN()**         Возвращает минимальное значение
- **MAX()**         Возвращает максимальное значение
    
2. *Ранжирующие*

- **ROW_NUMBER()**  Возвращает последовательность ранг (порядковый номер) строк внутри партиции, НЕЗАВИСИМО от того, есть ли в строках повторяющиеся значения или нет
- **RANK()**        Возвращает ранг каждой строки внутри партиции. Если есть повторяющиеся значения, функция возвращает одинаковый ранг для таких строчек, пропуская при этом следующий числовой ранг
- **DENSE_RANK()**  Возвращает ранг, но в случае одинаковых значений не пропускает следующий числовой ранг, а идет последовательно
- **NTILE()**       Возвращает номер группы, в которую попадает соответствующая строка результирующего набора
- **CUME_DIST()**   Возвращает кумулятивное распределение величины в пределах набора значений. Она представляет количество строк со значениями, меньшими или равными значению этой строки, деленному на общее количество строк

3. *Функции смещения*

- **LAG()**         Возвращает предыдущее значение столбца по порядку сортировки
- **LEAD()**        Возвращает следующее значение столбца по порядку сортировки
- **FIRST_VALUE()** Возвращает первое значение столбца в указанной партиции
- **LAST_VALUE()**  Возвращает последнее значение столбца в указанной партиции
- **NTH_VALUE(,n)** Возвращает n-ое значение в упорядоченном наборе значений из аналитического окна

### Примеры

    SELECT 
        p.*,
        ROW_NUMBER() OVER (PARTITION BY p.brand ORDER BY p.name) RN
    FROM products p;
    
    SELECT 
        product_id,
        start_date,
        price,
        MIN(price) OVER (PARTITION BY start_date) as min_price_date,
        MIN(price) OVER (PARTITION BY product_id) as min_price_product
    FROM prices p
    ORDER BY product_id, start_date ;

Аналитические функции вычисляются на итоговой выборке – только в SELECT или ORDER BY.

Если нужно отфильтровать по значению функции, то делайте подзапрос.
О подзапросах вы узнаете на следующем занятии.

# Источники

- https://thecode.media/mysql-uroot-p/
- https://sites.google.com/site/raznyeurokipoinformatiki/home/bazy-dannyh/teoria-po-bazam-dannyh/klassifikacia-baz-dannyh-po-srede-hranenia
- http://www.sql-tutorial.ru/ru/book_sql_dml.html

## Задания на SQL

Мы будем работать с базой данных о товарах и покупках в магазинах. Структура нашей базы данных следующая:

### Таблица products

Здесь содержится информация о товарах, продающихся в нашей торговой сети. Колонки:

- **product_id**: *INT* - числовой уникальный идентификатор товара в нашей БД
- **name**: *VARCHAR* - название товара
- **category**: *VARCHAR* - название категории товара
- **brand**: *BRAND* - название фирмы-производителя товара

### Таблица stores

Здесь содержится информация о магазинах нашей торговой сети. Колонки:

- **store_id**: *INT* - числовой уникальный идентификатор магазина в нашей БД
- **address**: *VARCHAR* - текстовый адрес магазина
- **region**: *INT* - числовой идентификатор региона, в котором находится магазин

### Таблица customers

Таблица, в которой содержится информация о зарегистрированных клиентах торговой сети. Колонки:

- **customer_id**: *INT* - числовой уникальный идентификатор клиента в БД
- **name**: *VARCHAR* - имя клиента
- **surname**: *VARCHAR* - фамилия клиента
- **birth_date**: *DATE* - дата рождения клиента

### Таблица prices

В этой таблице содержится информация о ценах на товары. Считаем что цена на конкретный товар одинакова в каждом из магазинов, но может меняться во времени. Поэтому цены мы храним в формате товар - дата начала действия цены - дата окончания действия цены. Колонки этой таблицы:

- **price_id**: *INT* - числовой уникальный идентификатор записи о цене
- **product_id**: *INT* - числовой уникальный идентификатор товара в нашей БД - отсылка к таблице **products**
- **price**: *INT* - стоимость товара в конкретный период в условных денежных единицах
- **start_date**: *DATE* - дата начала действия цены на товар (включительно)
- **end_date**: *DATE* - дата окончания действия цены на товар (включительно)

### Таблица sales

В этой таблице содержатся данные о продажах товаров в магазинах. Здесь сознательно не указывается цена, потому что цену продажи определенного товара в определенный день можно узнать из таблицы **prices**. Колонки:

- **sale_id**: *INT* - числовой уникальный идентификатор записи о продаже товара
- **product_id**: *INT* - числовой уникальный идентификатор товара в нашей БД - отсылка к таблице **products**
- **store_id**: *INT* - числовой уникальный идентификатор магазина в нашей БД - отсылка к таблице **stores**
- **customer_id**: *INT* - числовой уникальный идентификатор клиента в нашей БД - отсылка к таблице **customers**. Может быть NULL в случае, если клиент не авторизовался при покупке товара
- **sale_date**: *DATE* - дата продажи товара

В каждой задаче, описанной ниже, вам нужно работать только с одной из таблиц (задачи на объединение таблиц будут позже). Решение задачи - это скрипт, который выбирает нужные данные из таблицы. Его и нужно сохранить в файл с требуемым названием.

## Итак, задачи:

Сохраните запросы к каждой задаче в файлы src/01.sql, src/02.sql и т.д., в соответствии с номером задания. При проверке придется исполнять запросы пиров.

### Базовые запросы 
 
1. Выберите все адреса магазинов, которые хранятся в БД. 
2. Выберите все уникальные номера регионов магазинов. 
3. Выберите все уникальные сочетания имя-фамилия зарегистрированных клиентов. 
4. Мы хотим создать карту, в какие магазины ходит каждый конкретный покупатель. Выберите все пары customer_id, store_id для этого. 
5. Выведите все уникальные бренды товаров в алфавитном порядке. 
6. Мы хотим оценить, какой объем персональных предложений планировать на следующий год. Выведите customer_id и дату рождения клиентов, отсортированные по дате рождения без учета года (от 1 января до 31 декабря). 
7. Мы разделили регионы по возрастанию номера региона на тройки, чтобы провести на них проверку бизнес-гипотез. Выведите тертью тройку номеров регионов. 
8. Выведите самую позднюю дату, на которую хотя бы для одного товара установлена цена. Для решения воспользуйтесь базовыми операторами. 
 
 
### Запросы на WHERE 
 
9. Выведите все уникальные имена клиентов, фамилия которых Джигурда. 
10. Выведите все уникальные адреса магазинов региона 5 в алфавитном порядке. 
11. Выведите все уникальные product_id, которые были проданы покупателю с customer_id = 69 14 февраля 2020 года. 
12. Выведите все пары product_id - store_id, которые купили зарегистрированные пользователи 14 февраля 2020 года. 
13. Выведите все данные по ценам, которые действовали в течение февраля 2020 года (учтите оба поля start_date и end_date). 
14. Мы хотим сделать акцию к международному дню буквы К и оценить, сколько покупателей могут праздновать этот день. Выведите все customer_id, имена и фамилии покупателей, у которых имя или фамилия начинаются с буквы К (кириллицей). 
15. Выведите все уникальные customer_id покупателей, которые совершали покупки в феврале 2020 года, в порядке возрастания. 
 
 
### Запросы на агрегирующие функции и GROUP BY 
 
16. Посчитайте количество магазинов в регионе 5. Выведите единственное число. 
17. Выведите средннюю цену каждого товара за 2020 год в виде product_id - avg_price, где avg_price - это название колонки со средней ценой. Отсортируйте выборку в порядке возрастания product_id. 
18. Мы хотим изучить, в какие дни у нас было больше всего продаж. Выведите пары дата - количество продаж в те дни, когда их было больше 186000. 
19. Выведите количество дней, в которые количество продаж было больше 186000. 
20. Выведите количество продаж по месяцам в 2020 году. Формат вывода: двузначный номер месяца - количество продаж. 
21. Выведите самый популярный product_id среди незарегистрированных клиентов. 
22. Посчитайте количество однофамильцев для каждой фамилии. Выведите колонки surname, n, где n - это название колонки с количеством людей. 
